In [1]:
### 사용할 라이브러리
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

In [2]:
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [3]:
#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

검색할 키워드를 입력해주세요:건강기능식품

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):20

크롤링할 종료 페이지:  20 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=건강기능식품&start=91', 'ht

100%|████████████████████████████████████████████████████████████████████████████████████████| 311/311 [00:00<?, ?it/s]


In [4]:
# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")
    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
        
    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

100%|████████████████████████████████████████████████████████████████████████████████| 111/111 [01:12<00:00,  1.52it/s]

검색된 기사 갯수: 총  200 개

[뉴스 제목]
['눈 건강 기능식품 \'루테인\' 가격 천차만별…"최대 4배차"', "'눈 건강' 루테인 건강기능식품, 가격 최대 4배 차이", '대웅바이오, 건강기능식품 출사표…"3년 내 매출 1000억원"', '동아, 어린이 눈 건강기능식품 출시…"기능성원료 함유"', '삼진제약, 민감피부 개선 건강기능식품 출시', '차 종합연구원·비티씨 공동연구…"건강 기능 소재 개발"', '백종헌 “건강기능식품 표시·광고 심의위, 법률위원 절반은 불참”', '[포토]건기식협회 연구발표회… 홍지연 고려대 교수 ‘건강기능식품 사회적비용 절감 효과’ 발표', "한국건강기능식품협회 '건강기능식품 섭취에 따른 사회경제적 비용 절감 효과 연구발표회' 성료", '일동제약, 약국용 건강기능식품 일동 액상 마그네슘 출시', '한국건강기능식품협회 연구발표회 전경', '삼양그룹, 건강식품·친환경 소재로 100년 기업 도약', '삼양사, 스페셜티 소재로 일본 건강식품원료 시장 공략', '[포토]건기식협회 ‘건강기능식품 사회적비용 절감 효과’ 발표회… 지인배 동국대 교수 주제발표', '[농업이 IT(잇)다] 쿼드비 “프로젝트라이프로 맞춤형 기능성 식품 전개"', '“건강기능식품으로 300% 수익”…4000억 투자사기 일당 검거', '[에듀플러스]선배에게 듣는 진학상담〈27〉경희대 한약학과 “한약사 취득 후 한약국 운영·천연물 활용 건강식품 연구 등 진로 다양”', '"\'내가 먹어보니\' 하지 마세요"…건강기능식품 광고, \'체험기\'는 안된다', '“선진국에선 생명수”…가짜 건강기능식품 4000억 폰지사기 일당 검거', '홈쇼핑 방송 건강기능식품 판매 5년새 2배 증가…위반사례는 310건', '“건강기능식품으로 300% 수익” 투자자 속여 4천억 뜯어냈다 적발', '‘몸에 좋다는데’…올바른 건강기능식품 구매하려면', '쎈타임 `쏘팔메토 업`, `2023 한국소비자평가대상` 남성건강기능식품 부문 수상', '앙팡 다이노비타민C, ‘2023 KTCA한국소비자

In [5]:
###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
news_df

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

중복 제거 후 행 개수:  252


In [6]:
news_df

,date,title,link,content
0,2023-10-02 19:01:01,“건강기능식품으로 300% 수익”…4000억 투자사기 일당 검거,https://n.news.naver.com/mnews/article/011/000...,"[\n""투자하면 수익 300% 보장"" 홍보홍보한 '플빅산'은 비허가 액상 비료폰지사..."
1,2023-10-02 17:05:01,“선진국에선 생명수”…가짜 건강기능식품 4000억 폰지사기 일당 검거,https://n.news.naver.com/mnews/article/005/000...,[\n\n\n\n\n고모씨 일당이 원금의 최대 3배 이상의 수익을 내줄 수 있는 건...
2,2023-10-02 16:02:00,"""'내가 먹어보니' 하지 마세요""…건강기능식품 광고, '체험기'는 안된다",https://n.news.naver.com/mnews/article/057/000...,[\n10년 이하의 징역·1억 원 이하의 벌금\n\n\n\n사진=연합뉴스최근 조국 ...
3,2023-10-02 12:01:32,“건강기능식품으로 300% 수익” 투자자 속여 4천억 뜯어냈다 적발,https://n.news.naver.com/mnews/article/056/001...,[\n\n\n\n\n건강기능식품으로 수익을 낼 수 있다고 속여 3천 명이 넘는 투자...
4,2023-10-02 18:43:04,"""300% 이익"" 비료로 만든 건강식품 홍보해 4천억 챙긴 일당 무더기 송치",https://n.news.naver.com/mnews/article/214/000...,[\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n[뉴스데스크] ◀ 앵커...
...,...,...,...,...
247,2023-07-24 16:55:04,"정명수 한국건강기능식품협회장, 마약범죄 예방 캠페인 'NO EXIT 챌린지' 동참",https://n.news.naver.com/mnews/article/076/000...,[\n\n\n\n\n정명수 한국건강기능식품협회장이 마약범죄 예방 캠페인 'NO EX...
248,2023-09-27 14:02:14,"정관장, '랩' 브랜드 론칭..고함량 비타민C·D 제품 선보인다",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n맥스 비타민C 2000㎎과 맥스 비타민D 4000IU /사진=K...
249,2023-07-13 08:58:00,"건강기능식품 비비랩, 전면 리브랜딩…신규 슬로건 '뷰티베이스' 발표",https://n.news.naver.com/mnews/article/421/000...,[\n\n\n\n\n사진제공=㈜뉴트리원(서울=뉴스1) 장도민 기자 = 건강기능식품 ...
250,2023-09-21 09:14:09,추석 성수품 취급업소서 '미국 쇠고기→한우' 둔갑,https://n.news.naver.com/mnews/article/003/001...,"[\n부산시 특사경, 140곳 대상 특별단속…10건 적발\n\n\n\n[부산=뉴시스..."
